In [ ]:
# Para Politikası Metinleri: Şahinlik/Güvercinlik Analizi ve Kelime Ağı
# Gereken kütüphaneler:
# !pip install pdfplumber transformers torch nltk pyvis networkx pandas matplotlib scikit-learn 


import pdfplumber, re, os, string, pandas as pd, networkx as nx
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from pyvis.network import Network
import matplotlib.pyplot as plt

# --- 1. PDF'den Metin Çıkarma ---
pdf_dir = "/Users/xxxxxxx/Desktop/python2025/xxxxxxx"  # PDF klasörünü buraya koyun
date_pattern = re.compile(r"(\d{1,2}\s+(Ocak|Şubat|Mart|Nisan|Mayıs|Haziran|Temmuz|Ağustos|Eylül|Ekim|Kasım|Aralık)\s+\d{4})", re.IGNORECASE)

data = []
for file in tqdm(os.listdir(pdf_dir)):
    if not file.endswith(".pdf"): continue
    file_path = os.path.join(pdf_dir, file)
    with pdfplumber.open(file_path) as pdf:
        text = "\n".join(page.extract_text() or "" for page in pdf.pages)
    date_match = date_pattern.search(text)
    date_text = date_match.group(1) if date_match else None
    data.append({"file": file, "date": date_text, "text": text})

df = pd.DataFrame(data)

# --- 2. Temizleme ---
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation + "0123456789"))
    return text

df["clean_text"] = df["text"].apply(preprocess_text)

month_map = {
    "ocak": 1, "şubat": 2, "mart": 3, "nisan": 4, "mayıs": 5, "haziran": 6,
    "temmuz": 7, "ağustos": 8, "eylül": 9, "ekim": 10, "kasım": 11, "aralık": 12
}
def parse_date(date_str):
    if not date_str: return None
    parts = date_str.split()
    if len(parts) == 3:
        day = int(parts[0]); month = month_map.get(parts[1].lower(), None); year = int(parts[2])
        if month: return pd.Timestamp(year, month, day)
    return None
df["parsed_date"] = df["date"].apply(parse_date)
df = df.sort_values(by="parsed_date")

# --- 3. Sözlük Tabanlı Şahinlik Endeksi ---
hawkish_words = ["sıkılaştırma","faiz artışı","faiz artırımı","enflasyon","enflasyonist","fiyat istikrarı","risk","baskı","sıkı","oynaklık","enflasyon beklentisi","tedbir","önlem","parasal sıkılaştırma"]
dovish_words = ["gevşeme","faiz indirimi","faiz düşüşü","destekleyici","büyüme","istihdam","toparlanma","kredi genişlemesi","canlandırma","teşvik","parasal genişleme","rehberlik","güven artırıcı"]
def compute_hawkishness(text):
    words = text.split()
    hawk_count = sum(word in words for word in hawkish_words)
    dove_count = sum(word in words for word in dovish_words)
    total = hawk_count + dove_count
    return hawk_count / total if total > 0 else None
df["hawkish_score_dict"] = df["clean_text"].apply(compute_hawkishness)

# --- 4. BERTurk ile Şahinlik Skoru (0-1) ---
model_name = "dbmdz/bert-base-turkish-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()[0]

hawk_ref = "Bu metin sıkılaştırıcı bir para politikası tonuna sahiptir."
dove_ref = "Bu metin destekleyici ve gevşetici bir para politikası tonuna sahiptir."
hawk_emb = get_embedding(hawk_ref)
dove_emb = get_embedding(dove_ref)

bert_scores = []
for text in tqdm(df["clean_text"].fillna("").tolist()):
    if len(text.strip()) == 0:
        bert_scores.append(None); continue
    emb = get_embedding(text[:2000])
    sim_hawk = cosine_similarity([emb], [hawk_emb])[0][0]
    sim_dove = cosine_similarity([emb], [dove_emb])[0][0]
    score = sim_hawk / (sim_hawk + sim_dove) if (sim_hawk + sim_dove) > 0 else None
    bert_scores.append(score)
df["hawkish_score_bert"] = bert_scores

# --- 5. Zaman Serisi Çizimi ---
plt.figure(figsize=(10,5))
plt.plot(df["parsed_date"], df["hawkish_score_dict"], label="Sözlük Tabanlı", alpha=0.7)
plt.plot(df["parsed_date"], df["hawkish_score_bert"], label="BERT Tabanlı", alpha=0.7)
plt.legend(); plt.title("Şahinlik Endeksi (Sözlük vs BERT)"); plt.xlabel("Tarih"); plt.ylabel("Endeks"); plt.show()

# --- 6. Kelime Ağı ---
from collections import Counter
from itertools import combinations
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('turkish'))

def get_top_words(texts, top_n=300):
    all_words = []
    for t in texts:
        all_words.extend([w for w in t.split() if w not in stop_words and len(w)>3])
    return [w for w,_ in Counter(all_words).most_common(top_n)]

top_words = get_top_words(df["clean_text"].tolist(), top_n=300)